In [23]:
import json
import os

1. load subqa datasets

2. generate with Llama2

3. evaluate with GPT

4. make a loop

output format



### 1. load sub-QAs

In [24]:
data_path = "data/dev_subqa.json"

In [25]:
data = json.load(open(data_path))

In [26]:
len(data)

114

In [27]:
data[0]['SubQA']["DQ_1"]

{'question': 'What is an oligopoly?',
 'answer': 'An oligopoly is a market structure characterized by a small number of firms that control a large majority of the market share.'}

In [28]:
# def format_subject(subject):
#     l = subject.split("_")
#     s = ""
#     for entry in l:
#         s += " " + entry
#     return s

# def gen_prompt(subject, k=-1):
#     prompt = "The following are multiple choice questions (with answers) about {}.\n\n".format(format_subject(subject))
    
#     return prompt

# def format_example(ins, include_answer=True):
#     prompt = df.iloc[idx, 0]
#     k = df.shape[1] - 2
#     for j in range(k):
#         prompt += "\n{}. {}".format(choices[j], df.iloc[idx, j+1])
#     prompt += "\nAnswer:"
#     if include_answer:
#         prompt += " {}\n\n".format(df.iloc[idx, k + 1])
#     return prompt

## 2. generate with Llama2

In [15]:
from tqdm import tqdm

import torch
import transformers
from transformers import AutoModelForCausalLM, LlamaTokenizer


In [16]:
os.environ['CUDA_VISIBLE_DEVICES'] = '10'

In [17]:
ckpt_dir = "/home/gangwoo_kim/git-repos/mmlu/diff_ins/results/llama2-7b/sft"
# ckpt_dir = "meta-llama/Llama-2-7b-hf"

In [18]:
tokenizer = LlamaTokenizer.from_pretrained(
        ckpt_dir,
        use_fast=False,
        padding_side="left",
    )
tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id
tokenizer.bos_token_id = 1

In [20]:
model = AutoModelForCausalLM.from_pretrained(ckpt_dir, device_map = 'balanced_low_0', torch_dtype=torch.float16, trust_remote_code=True)
model.bfloat16()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.50s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [21]:
def prepare_input(tokenizer, prompts):
    input_tokens = tokenizer.batch_encode_plus(prompts, return_tensors="pt", padding=True)
    for t in input_tokens:
        if torch.is_tensor(input_tokens[t]):
            input_tokens[t] = input_tokens[t].to('cuda')

    return input_tokens

def batch_split(prompts, batch_num):
    batch_prompts = []
    mini_batch = []
    for prompt in prompts:
        mini_batch.append(prompt)
        if len(mini_batch) == batch_num:
            batch_prompts.append(mini_batch)
            mini_batch = []
    if len(mini_batch) != 0:
        batch_prompts.append(mini_batch)
    return batch_prompts

def batch_infer(model, tokenizer, prompts):
    batch_size = 4
    answers = []
    for batch_input in tqdm(batch_split(prompts, batch_size)):
        encode_inputs = prepare_input(tokenizer, batch_input)
        outputs = model.generate(**encode_inputs, max_new_tokens=100)
        answers.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
    # answers = [answer[-1] for answer in answers]
    return answers

In [48]:
subqs = []
subas = []
all_ins = []
# prompt = """
# Please provide a free-form response to the question in no more than 3 sentences.

# Question: What is the significance of having a few large producers in an oligopoly?
# Answer: The few large producers in an oligopoly hold a significant portion of the market share, giving them considerable market power to influence prices, production levels, and other market factors.
           
# ===============
# Question: 
# """
prompt = "please provide an answer to the question.\nQuestion: "

for entry in data:
    for i in range(1, 6):
        try:
            subq = prompt + entry['SubQA'][f"DQ_{i}"]["question"] + "\nAnswer: "
            suba = entry['SubQA'][f"DQ_{i}"]["answer"]
            ins = {"id" : entry["subject"] + "_dev_" + f"DQ_{i}", 
                "prompt" : subq, 
                "question" : entry['SubQA'][f"DQ_{i}"]["question"],
                "answer" : suba,
            }
            subqs.append(subq)
            subas.append(suba)
            all_ins.append(ins)
        except:
            pass



In [32]:
preds = batch_infer(model, tokenizer, subqs)

100%|██████████| 143/143 [10:07<00:00,  4.25s/it]


In [35]:
with open("subqa_results.json", 'w') as f:
    json.dump(preds, f)

In [33]:
print(preds[0])

please provide an answer to the question.
Question: What is an oligopoly?
Answer:  An oligopoly is an industry with only a few firms. These firms are so large that they almost monopolize the industry. They are called "oligopolists". Because there are only a few firms, there is very little competition. As a result, these firms can easily make agreements with each other. They can also easily influence each other's prices and output. In an oligopoly, a change in the market share of any one firm


## 3. Evaluate with GPT

In [36]:
from string import Template
from openai import OpenAI

In [37]:
OPENAI_API_KEY= # OpenAI API Key
model_name="gpt-3.5-turbo"

In [38]:
input_txt = "please provide an answer to the question.\nQuestion: How is market concentration measured in terms of firms?"

In [51]:
system_prompt = """
Given the two responses below, determine whether they describe the same factual information. Analyze the main points made in each response and assess their relevance and agreement to each other. Focus on the content of the facts presented, ignoring stylistic differences unless they impact the factual interpretation. Respond with ‘[TRUE]’ if they describe the same fact, or ‘[FALSE]’ if they do not.

Follow the following format

Question: ${question to be answered}
Reference Response: ${reference answer from knowledgeable teacher}
Generated Response: ${generated answer by student}
Analysis: ${a detailed comparison for each key point discussed in both responses}
Conclusion: ${the final decision as either ‘[TRUE]’ or ‘[FALSE]’}
"""

prompt = Template("""
Question: $question
Reference Response: $ref
Generated Response: $pred
Analysis:
""")

In [52]:
pred = preds[0].split("\nAnswer:")[-1].strip()
pred

'An oligopoly is an industry with only a few firms. These firms are so large that they almost monopolize the industry. They are called "oligopolists". Because there are only a few firms, there is very little competition. As a result, these firms can easily make agreements with each other. They can also easily influence each other\'s prices and output. In an oligopoly, a change in the market share of any one firm'

In [53]:
all_ins[0]

{'id': 'high_school_microeconomics_dev_DQ_1',
 'prompt': 'please provide an answer to the question.\nQuestion: What is an oligopoly?\nAnswer: ',
 'question': 'What is an oligopoly?',
 'answer': 'An oligopoly is a market structure characterized by a small number of firms that control a large majority of the market share.'}

In [54]:
question = all_ins[0]['question']
ref      = all_ins[0]['answer']

In [67]:
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

def val_answer(client, question, ref, pred):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": prompt.substitute(question=question, ref=ref, pred=pred),
            }
        ],
        model=model_name,
    )
    try:
        response = chat_completion.choices[0].message.content
        
        analysis, result = response.split("Conclusion:")
        analysis = analysis.split("Analysis:")[-1].strip()
        result = result.strip() == "[TRUE]"
    except:
        analysis, result = -1, -1
        
    return response, analysis, result

In [68]:
val_answer(client, question, ref, pred)

("Analysis: \nBoth responses discuss Max Weber's contribution to sociology. The reference response highlights Weber's analysis of the relationship between economic structures and legal systems as one of his significant contributions. On the other hand, the generated response focuses on Weber being one of the first individuals to approach sociology as a science, highlighting the disorder in the field before his work. While the generated response focuses more on Weber's efforts to establish sociology as a disciplined science, it does not directly mention his specific analyses. However, both responses highlight Weber's pivotal role in the development of sociology.\nConclusion: [FALSE]",
 "Both responses discuss Max Weber's contribution to sociology. The reference response highlights Weber's analysis of the relationship between economic structures and legal systems as one of his significant contributions. On the other hand, the generated response focuses on Weber being one of the first ind

In [86]:
for ins, pred in zip(all_ins, preds):
    question = ins['question']
    ref      = ins['answer']
    res, analy, result = val_answer(client, question, ref, pred)
    
    ins["response"] = res
    ins["correct"] = result

TypeError: 'zip' object is not subscriptable

In [87]:
all_ins[10]

{'id': 'conceptual_physics_dev_DQ_1',
 'prompt': 'please provide an answer to the question.\nQuestion: What is the basic structure of an atom?\nAnswer: ',
 'question': 'What is the basic structure of an atom?',
 'answer': 'An atom consists of a nucleus (made up of protons and neutrons) and electrons orbiting around the nucleus, with the mass primarily concentrated in the nucleus.',
 'response': 'Reference Response: An atom consists of a nucleus (made up of protons and neutrons) and electrons orbiting around the nucleus, with the mass primarily concentrated in the nucleus.\nGenerated Response: The basic structure of an atom includes a nucleus containing protons and neutrons, with electrons orbiting around the nucleus, with the majority of the mass located in the nucleus.\n\nAnalysis: Both responses correctly identify the basic structure of an atom, mentioning the nucleus composed of protons and neutrons, with electrons orbiting around it. They both also note that the majority of the mas

In [89]:
cors = []

for ins in all_ins:
    if "correct" not in ins.keys():
        break
    
    cors.append(ins["correct"])

In [90]:
len(cors)

436

In [91]:
sum(cors) / len(cors)

0.3876146788990826

In [92]:
cors[:5]

[True, False, False, True, True]

In [104]:
i = 3

In [105]:
data[i]

{'question': 'Colors in a soap bubble result from light',
 'subject': 'conceptual_physics',
 'choices': ['converted to a different frequency',
  'deflection',
  'interference',
  'polarization'],
 'answer': 2,
 'explanation': "Colors in a soap bubble result from light interference. This phenomenon occurs when light waves that are reflected off the outer and inner surfaces of the soap bubble's thin film overlap. A soap bubble consists of a thin layer of water sandwiched between two layers of soap molecules. When light hits the bubble, some of it is reflected off the outer surface, and some penetrates the bubble and is reflected off the inner surface. Because the light that penetrates has to travel a slightly longer path, the waves of light are not perfectly in sync when they recombine. This slight difference in path length causes the waves to interfere with each other.\n\nInterference can be constructive or destructive, depending on the phase difference between the waves. Constructive i

In [106]:
for ins in all_ins[i*5:i*5 + 5]:
    print("\n")
    print(ins['id'])
    print(ins['response'])
    print(ins['correct'])



conceptual_physics_dev_DQ_1
Question: What is the structure of a soap bubble that affects how light interacts with it?
Reference Response: A soap bubble consists of a thin layer of water sandwiched between two layers of soap molecules.
Generated Response: 3
Analysis: The reference response correctly describes the structure of a soap bubble, highlighting that it is composed of a thin layer of water with two layers of soap molecules. On the other hand, the generated response is unrelated and simply states "3" without providing any relevant information about the structure of a soap bubble.
Conclusion: [FALSE]
False


conceptual_physics_dev_DQ_2
Question: How does light behave when it encounters a soap bubble?
Reference Response: When light hits a soap bubble, some of it is reflected off the outer surface, and some penetrates the bubble and is reflected off the inner surface.
Generated Response: Light will refract through the bubble. Light will pass through the bubble.

Analysis: 
- The 

In [ ]:
res

'Question: Why is specificity in terms such as price important for an offer to be considered valid in contract law?\nReference Response: Specificity in terms such as price is crucial because it allows for the formation of a contract with clear terms, enabling both parties to understand their obligations and rights, and reduces ambiguity that could lead to disputes.\nGenerated Response: If an offer is too general, then the offeree must add something to make it definite. If the offeree does this, the original offeror may not accept the offer. Specificity is important because it avoids misunderstandings between the parties as to what is being offered and prevents one party from offering something different from what was agreed upon. For example, Party A offers Party B $10,000 "for the car." Party B responds by saying "$10,000 for the blue car parked outside your house," which clarifies the terms of the offer. This specificity helps in avoiding ambiguity and ensuring both parties are on th

In [135]:
tok = tokenizer(input_txt, return_tensors="pt", padding=True)
tok.to("cuda")

{'input_ids': tensor([[    1,  3113,  3867,   385,  1234,   304,   278,  1139, 29889,    13,
         16492, 29901,  1128,   338,  9999, 26702, 17005,   297,  4958,   310,
         13734,  1516, 29973]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [ ]:
out = model.generate(tok.input_ids, max_new_tokens=1).cpu().tolist()

In [137]:
tokenizer.decode(out[0], skip_special_tokens=True)

'please provide an answer to the question.\nQuestion: How is market concentration measured in terms of firms?\n1. The number of firms in the industry.\n2. The number of firms in the industry.\n3. The number of firms in the industry.\n4. The number of firms in the industry.\n5. The number of firms in the industry.\n6. The number of firms in the industry.\n7. The number of firms in the industry.\n8. The number of firms in the industry.\n9. The'